In [1]:
import sqlite3
import os
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra

In [2]:
filename = "/global/cfs/cdirs/desi/science/td/db/desi.db"
con = sqlite3.connect(filename)

In [3]:
cur = con.cursor()

## Note that the following doesn't list multiple exposures of the same tile the same night

In [4]:
targetid_q = """
select distinct mtl.targetid, f.yyyymmdd, f.tile, f.petal
from mtl
inner join fibermap_daily f
on mtl.targetid==f.targetid
where mtl.program='ToO'
order by mtl.targetid, f.yyyymmdd, f.tile, f.petal ;
"""

In [5]:
cur.execute(targetid_q)
q_arr = cur.fetchall()
cur.close()

## Produce list that groups tile/petal combinations observed for the same targetid the same night

In [6]:
ans =[]
tid_night=[q_arr[0]]
for i in range(1,len(q_arr)):
    if q_arr[i][0]== tid_night[0][0] and q_arr[i][1]== tid_night[0][1]:
        tid_night.append(q_arr[i])
    else:
        ans.append(tid_night)
        tid_night=[q_arr[i]]
        if i == len(q_arr)-1:
             ans.append(tid_night)

In [7]:
datapath = os.path.join(os.environ.get('DESI_REDUX'),'daily','tiles','cumulative')

In [13]:
for a in ans:
    spectocoadd=[]
    for targetid, yyyymmdd, tile, petal in a:
        filename = os.path.join(datapath,str(tile),str(yyyymmdd),f"spectra-{petal}-{tile}-thru{yyyymmdd}.fits")
        spec = read_spectra(filename)
        spec = spec.select(targets=[targetid])
        print(spec.num_spectra())
        weffe

INFO:spectra.py:253:read_spectra: iotime 0.799 sec to read spectra-0-233-thru20210504.fits at 2021-05-26T11:46:38.731812
1


NameError: name 'weffe' is not defined

(43977540930371976, 20210506, 290, 5)
